In [1]:
import sqlite3
import pandas as pd

In [2]:
# Read the code below and write your observation in the next cell

conn = sqlite3.connect('eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


In [3]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

num
name
content


In [4]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [5]:
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])
    
    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [6]:
from tqdm import tqdm
tqdm.pandas()

df['file_content'] = df['content'].progress_apply(decode_method)

# Display the first few rows of the DataFrame
df.head()

100%|███████████████████████████████████████████████████████████████████████████| 82498/82498 [01:29<00:00, 917.37it/s]


,num,name,content,file_content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...,"1\r\n00:00:29,359 --> 00:00:32,048\r\nAh! Ther..."
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...,"1\r\n00:00:53,200 --> 00:00:56,030\r\n<i>Yumi'..."
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."


In [7]:
df_sample= df.sample(frac=0.3)

import re
# Regular expression pattern to remove timestamps
timestamp_pattern = r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}'
# Regular expression pattern to remove unwanted characters like '\r', '\n', 'ï»¿'
unwanted_pattern = r'[\r\nï»¿]'

# Function to clean the content
def clean_content(text):
    text = re.sub(timestamp_pattern, '', text)  # Remove timestamps
    text = re.sub(unwanted_pattern, '', text)    # Remove unwanted characters
    text = re.sub(r'<.*?>', '', text)            # Remove HTML tags
    text = text.strip()                          # Strip leading/trailing whitespace
    text = re.sub(r'^\d+', '', text.strip())     # Remove leading '1'
    return text.strip()

# Apply the cleaning function to the 'content' column
df_sample['cleaned_content'] = df_sample['file_content'].progress_apply(clean_content) 

100%|███████████████████████████████████████████████████████████████████████████| 24749/24749 [03:59<00:00, 103.19it/s]


In [7]:
df_sample= df.sample(frac=0.3)

import re
# Regular expression pattern to remove timestamps
timestamp_pattern = r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}'
# Regular expression pattern to remove unwanted characters like '\r', '\n', 'ï»¿'
unwanted_pattern = r'[\r\nï»¿]'

# Function to clean the content
def clean_content(text):
    text = re.sub(timestamp_pattern, '', text)  # Remove timestamps
    text = re.sub(unwanted_pattern, '', text)    # Remove unwanted characters
    text = re.sub(r'<.*?>', '', text)            # Remove HTML tags
    text = text.strip()                          # Strip leading/trailing whitespace
    text = re.sub(r'^\d+', '', text.strip())     # Remove leading '1'
    return text.strip()

# Apply the cleaning function to the 'content' column
df_sample['cleaned_content'] = df_sample['file_content'].progress_apply(clean_content)



100%|███████████████████████████████████████████████████████████████████████████| 24749/24749 [03:59<00:00, 103.19it/s]


In [8]:
## step3: Data Preprocessing on train data
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [10]:
df_sample

,num,name,content,file_content,cleaned_content
59077,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00Q9\x9aV\x9...,"ï»¿1\r\n00:00:01,235 --> 00:00:04,903\r\nâª <...",âª It seems todaythat all you see âª2âª Is ...
55084,9412321,the.xfiles.s01.e16.young.at.heart.(1994).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x87.\x9aV...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nUse t...",Use the free code JOINNOW at â¨www.playships....
78286,9504869,jane.s01.e09.rangifer.tarandus.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xe8\x98\x...,"1\n00:00:41,416 --> 00:00:43,456\n""Rangifer ta...","""Rangifer tarandus.""2Jane, how long do we have..."
44076,9361962,house.of.the.dragon.s01.e06.the.princess.and.t...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\'\x1b\x9a...,"ï»¿1\r\n00:02:00,706 --> 00:02:02,582\r\nKeep ...",Keep breathing.2And push.3And again.4I can't.5...
45370,9369763,the.cleaner.s02.e04.a.clean.christmas.(2022).e...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00(\x12\x9aV...,"ï»¿1\r\n00:00:03,880 --> 00:00:10,760\r\nThis ...",This programme contains somestrong language an...
...,...,...,...,...,...
29035,9297376,love.live.superstar.s01.e08.connecting.feeling...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1d\xa8\x...,[Script Info]\r\nTitle: English (US)\r\nOrigin...,[Script Info]Title: English (US)Original Scrip...
34430,9318197,medium.s04.e06.aftertaste.(2008).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00M\xb1\x99V...,"ï»¿1\r\n00:00:02,319 --> 00:00:05,454\r\nMmm.....","Mmm...2Oh, my God.3This is good.4(classical mu..."
26461,9286615,the.dresden.files.s01.e07.walls.(2007).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00T\xa4\x99V...,"ï»¿1\r\n00:00:03,170 --> 00:00:06,583\r\n<i>(D...",(Dresden narrating) MY FATHER USEDTO SAY OUR L...
69236,9466003,the.jeffersons.s10.e16.my.guy.george.(1984).en...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xd8U\x9aV...,"ï»¿1\r\n00:00:07,697 --> 00:00:11,632\r\nin th...",in the sky2a piece of the pie3in the kitchen4o...


In [9]:
def preprocess(raw_text):
    # Removing special characters and digits
    sentence = re.sub("[^a-zA-Z]", " ", raw_text)

    #change sentence to lower case
    sentence = sentence.lower()

    # tokenize into words
    tokens =nltk.word_tokenize(sentence)

    #Lemmatize
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    #remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in lemmatized_tokens if word.lower() not in stop_words]

    #Join and return
    return " ".join(filtered_tokens)

In [11]:
df_sample['cleaned_content'] = df_sample['cleaned_content'].progress_apply(lambda x:  preprocess(x))

100%|████████████████████████████████████████████████████████████████████████████| 24749/24749 [29:38<00:00, 13.92it/s]


In [12]:
df_sample

,num,name,content,file_content,cleaned_content
59077,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00Q9\x9aV\x9...,"ï»¿1\r\n00:00:01,235 --> 00:00:04,903\r\nâª <...",seems todaythat see violence moviesand sex tv ...
55084,9412321,the.xfiles.s01.e16.young.at.heart.(1994).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x87.\x9aV...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nUse t...",use free code joinnow www playships eu god you...
78286,9504869,jane.s01.e09.rangifer.tarandus.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xe8\x98\x...,"1\n00:00:41,416 --> 00:00:43,456\n""Rangifer ta...",rangifer tarandus jane long hide cold long tak...
44076,9361962,house.of.the.dragon.s01.e06.the.princess.and.t...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\'\x1b\x9a...,"ï»¿1\r\n00:02:00,706 --> 00:02:02,582\r\nKeep ...",keep breathing push push head boy princess pra...
45370,9369763,the.cleaner.s02.e04.a.clean.christmas.(2022).e...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00(\x12\x9aV...,"ï»¿1\r\n00:00:03,880 --> 00:00:10,760\r\nThis ...",programme contains somestrong language adult h...
...,...,...,...,...,...
29035,9297376,love.live.superstar.s01.e08.connecting.feeling...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1d\xa8\x...,[Script Info]\r\nTitle: English (US)\r\nOrigin...,script info title english u original script ba...
34430,9318197,medium.s04.e06.aftertaste.(2008).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00M\xb1\x99V...,"ï»¿1\r\n00:00:02,319 --> 00:00:05,454\r\nMmm.....",mmm oh god good classical musicplaying quietly...
26461,9286615,the.dresden.files.s01.e07.walls.(2007).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00T\xa4\x99V...,"ï»¿1\r\n00:00:03,170 --> 00:00:06,583\r\n<i>(D...",dresden narrating father usedto say limitation...
69236,9466003,the.jeffersons.s10.e16.my.guy.george.(1984).en...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xd8U\x9aV...,"ï»¿1\r\n00:00:07,697 --> 00:00:11,632\r\nin th...",sky piece pie kitchen grill big league baby sk...


In [13]:
def chunk_document(corpous, id_col, chunk_size=300):
    data = []

    for doc,id  in zip(corpous, id_col):
        
        words = doc.split()
        for i in range(0, len(words), chunk_size):
            chunk = ' '.join(words[i:i + chunk_size])


            data.append((id,chunk))

    df = pd.DataFrame(data)

    return df

In [14]:
df_sample

,num,name,content,file_content,cleaned_content
59077,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00Q9\x9aV\x9...,"ï»¿1\r\n00:00:01,235 --> 00:00:04,903\r\nâª <...",seems todaythat see violence moviesand sex tv ...
55084,9412321,the.xfiles.s01.e16.young.at.heart.(1994).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x87.\x9aV...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nUse t...",use free code joinnow www playships eu god you...
78286,9504869,jane.s01.e09.rangifer.tarandus.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xe8\x98\x...,"1\n00:00:41,416 --> 00:00:43,456\n""Rangifer ta...",rangifer tarandus jane long hide cold long tak...
44076,9361962,house.of.the.dragon.s01.e06.the.princess.and.t...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\'\x1b\x9a...,"ï»¿1\r\n00:02:00,706 --> 00:02:02,582\r\nKeep ...",keep breathing push push head boy princess pra...
45370,9369763,the.cleaner.s02.e04.a.clean.christmas.(2022).e...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00(\x12\x9aV...,"ï»¿1\r\n00:00:03,880 --> 00:00:10,760\r\nThis ...",programme contains somestrong language adult h...
...,...,...,...,...,...
29035,9297376,love.live.superstar.s01.e08.connecting.feeling...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1d\xa8\x...,[Script Info]\r\nTitle: English (US)\r\nOrigin...,script info title english u original script ba...
34430,9318197,medium.s04.e06.aftertaste.(2008).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00M\xb1\x99V...,"ï»¿1\r\n00:00:02,319 --> 00:00:05,454\r\nMmm.....",mmm oh god good classical musicplaying quietly...
26461,9286615,the.dresden.files.s01.e07.walls.(2007).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00T\xa4\x99V...,"ï»¿1\r\n00:00:03,170 --> 00:00:06,583\r\n<i>(D...",dresden narrating father usedto say limitation...
69236,9466003,the.jeffersons.s10.e16.my.guy.george.(1984).en...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xd8U\x9aV...,"ï»¿1\r\n00:00:07,697 --> 00:00:11,632\r\nin th...",sky piece pie kitchen grill big league baby sk...


In [15]:
chuncked_df = chunk_document(df_sample.cleaned_content,df_sample.num)

In [16]:
chuncked_df

,0,1
0,9428782,seems todaythat see violence moviesand sex tv ...
1,9428782,bike hi help u tryingto locate lost big wh ins...
2,9428782,quiz gon na give name tell nameof yankee candl...
3,9428782,brand feel isappropriate compensation et ceter...
4,9428782,thatif sigh right kind took advantage owe apol...
...,...,...
226170,9521232,r right away oh good good well great news good...
226171,9521232,harder jenny moaning gena gary gary gary like ...
226172,9521232,house need help could come get rescue got full...
226173,9521232,greedy kate care isbeing first market whether ...


In [17]:
chuncked_df.columns =['num','file_content_chunks']

In [18]:
chuncked_df

,num,file_content_chunks
0,9428782,seems todaythat see violence moviesand sex tv ...
1,9428782,bike hi help u tryingto locate lost big wh ins...
2,9428782,quiz gon na give name tell nameof yankee candl...
3,9428782,brand feel isappropriate compensation et ceter...
4,9428782,thatif sigh right kind took advantage owe apol...
...,...,...
226170,9521232,r right away oh good good well great news good...
226171,9521232,harder jenny moaning gena gary gary gary like ...
226172,9521232,house need help could come get rescue got full...
226173,9521232,greedy kate care isbeing first market whether ...


In [19]:
merged_df = chuncked_df.merge(df_sample,left_on="num",right_on="num",how="left")

In [20]:
merged_df

,num,file_content_chunks,name,content,file_content,cleaned_content
0,9428782,seems todaythat see violence moviesand sex tv ...,family.guy.s17.e17.island.adventure.(2019).eng...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00Q9\x9aV\x9...,"ï»¿1\r\n00:00:01,235 --> 00:00:04,903\r\nâª <...",seems todaythat see violence moviesand sex tv ...
1,9428782,bike hi help u tryingto locate lost big wh ins...,family.guy.s17.e17.island.adventure.(2019).eng...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00Q9\x9aV\x9...,"ï»¿1\r\n00:00:01,235 --> 00:00:04,903\r\nâª <...",seems todaythat see violence moviesand sex tv ...
2,9428782,quiz gon na give name tell nameof yankee candl...,family.guy.s17.e17.island.adventure.(2019).eng...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00Q9\x9aV\x9...,"ï»¿1\r\n00:00:01,235 --> 00:00:04,903\r\nâª <...",seems todaythat see violence moviesand sex tv ...
3,9428782,brand feel isappropriate compensation et ceter...,family.guy.s17.e17.island.adventure.(2019).eng...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00Q9\x9aV\x9...,"ï»¿1\r\n00:00:01,235 --> 00:00:04,903\r\nâª <...",seems todaythat see violence moviesand sex tv ...
4,9428782,thatif sigh right kind took advantage owe apol...,family.guy.s17.e17.island.adventure.(2019).eng...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00Q9\x9aV\x9...,"ï»¿1\r\n00:00:01,235 --> 00:00:04,903\r\nâª <...",seems todaythat see violence moviesand sex tv ...
...,...,...,...,...,...,...
226170,9521232,r right away oh good good well great news good...,workin.moms.s07.e08.fast.and.soft.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00W\x96\x9aV...,"ï»¿1\r\n00:00:07,130 --> 00:00:09,880\r\nâª\r...",jane anne still awake scootch day get young wo...
226171,9521232,harder jenny moaning gena gary gary gary like ...,workin.moms.s07.e08.fast.and.soft.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00W\x96\x9aV...,"ï»¿1\r\n00:00:07,130 --> 00:00:09,880\r\nâª\r...",jane anne still awake scootch day get young wo...
226172,9521232,house need help could come get rescue got full...,workin.moms.s07.e08.fast.and.soft.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00W\x96\x9aV...,"ï»¿1\r\n00:00:07,130 --> 00:00:09,880\r\nâª\r...",jane anne still awake scootch day get young wo...
226173,9521232,greedy kate care isbeing first market whether ...,workin.moms.s07.e08.fast.and.soft.(2023).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00W\x96\x9aV...,"ï»¿1\r\n00:00:07,130 --> 00:00:09,880\r\nâª\r...",jane anne still awake scootch day get young wo...


In [21]:
merged_df = merged_df[['num','name', 'file_content_chunks']]	

In [22]:
merged_df

,num,name,file_content_chunks
0,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,seems todaythat see violence moviesand sex tv ...
1,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,bike hi help u tryingto locate lost big wh ins...
2,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,quiz gon na give name tell nameof yankee candl...
3,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,brand feel isappropriate compensation et ceter...
4,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,thatif sigh right kind took advantage owe apol...
...,...,...,...
226170,9521232,workin.moms.s07.e08.fast.and.soft.(2023).eng.1cd,r right away oh good good well great news good...
226171,9521232,workin.moms.s07.e08.fast.and.soft.(2023).eng.1cd,harder jenny moaning gena gary gary gary like ...
226172,9521232,workin.moms.s07.e08.fast.and.soft.(2023).eng.1cd,house need help could come get rescue got full...
226173,9521232,workin.moms.s07.e08.fast.and.soft.(2023).eng.1cd,greedy kate care isbeing first market whether ...


In [28]:
for id, chunk in enumerate(merged_df):
    print(chunk)

num
name
file_content_chunks


In [24]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

In [23]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import chromadb

In [42]:
merged_df

,num,name,file_content_chunks
0,9211798,dr.stone.ryuusui.(2022).eng.1cd,THE KINGDOM OF SCIENCE'S FIRST PLAYTHE MYSTERI...
1,9211798,dr.stone.ryuusui.(2022).eng.1cd,two are usually so lazy...57but it looks liket...
2,9211798,dr.stone.ryuusui.(2022).eng.1cd,science and strength...108has got me so excite...
3,9211798,dr.stone.ryuusui.(2022).eng.1cd,as a kid...161so he collected models oftons of...
4,9211798,dr.stone.ryuusui.(2022).eng.1cd,ships.216How could a manually-operatedprimitiv...
...,...,...,...
331546,9285812,upstairs.downstairs.s01.e07.magic.casements.(1...,youoff in that new hat.721I'd be delighted to ...
331547,9285812,upstairs.downstairs.s01.e07.magic.casements.(1...,"morning room, sir.785Hammond.786You heard abou..."
331548,9285812,upstairs.downstairs.s01.e07.magic.casements.(1...,"just see her.840Well, at least there's nothing..."
331549,9285812,upstairs.downstairs.s01.e07.magic.casements.(1...,happened?899Charles...900I can't go on with it...


In [25]:
merged_df['doc_vector_pretrained_bert'] = merged_df.file_content_chunks.progress_apply(model.encode)

merged_df.head()

100%|███████████████████████████████████████████████████████████████████████| 226175/226175 [13:45:37<00:00,  4.57it/s]
C:\Users\Hello\AppData\Local\Temp\ipykernel_45592\2187253098.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df['doc_vector_pretrained_bert'] = merged_df.file_content_chunks.progress_apply(model.encode)


,num,name,file_content_chunks,doc_vector_pretrained_bert
0,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,seems todaythat see violence moviesand sex tv ...,"[-0.024769288, -0.07960797, 0.0017365585, -0.0..."
1,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,bike hi help u tryingto locate lost big wh ins...,"[-0.019719025, -0.04682111, 0.022851698, -0.02..."
2,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,quiz gon na give name tell nameof yankee candl...,"[-0.02157822, -0.10432916, 0.072285645, -0.026..."
3,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,brand feel isappropriate compensation et ceter...,"[-0.022212539, -0.11449277, 0.05176024, -0.076..."
4,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,thatif sigh right kind took advantage owe apol...,"[-0.021211743, -0.078733094, 0.018811794, -0.0..."


In [26]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226175 entries, 0 to 226174
Data columns (total 4 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   num                         226175 non-null  int64 
 1   name                        226175 non-null  object
 2   file_content_chunks         226175 non-null  object
 3   doc_vector_pretrained_bert  226175 non-null  object
dtypes: int64(1), object(3)
memory usage: 8.6+ MB


In [42]:
User_query = pd.Series([input("Enter a user Query " )])

Enter a user Query  peter


In [43]:

User_query.progress_apply(lambda x:  clean_content(x)) 

User_query.progress_apply(lambda x:  preprocess(x)) 

100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 200.49it/s]


0    peter
dtype: object

In [46]:
Query = User_query.progress_apply(model.encode)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 23.23it/s]


In [67]:
Query[0]

array([-2.16531549e-02,  4.10362966e-02, -1.03500835e-03, -2.97786091e-02,
       -6.63683042e-02, -9.52343550e-03,  5.62622100e-02,  3.15104425e-02,
       -2.29437146e-02, -3.30940373e-02, -6.69950396e-02, -4.52171043e-02,
       -9.01967287e-02, -3.01976991e-03, -1.97209488e-03, -1.07732890e-02,
       -7.68374801e-02,  5.28583005e-02,  1.70733035e-03, -9.42570642e-02,
       -1.29660293e-01, -1.79409180e-02,  2.80486345e-02, -4.87643257e-02,
        2.10065246e-02,  2.55819336e-02, -9.36805271e-03,  1.95052344e-02,
        1.09621994e-01, -1.16009109e-01, -1.70466397e-02,  8.89472850e-03,
       -2.86132880e-02, -5.01052178e-02, -2.82779653e-02, -6.98663585e-04,
       -6.15112968e-02,  1.11004747e-02, -2.52986941e-02,  1.76151283e-02,
        1.90327968e-02, -6.16267184e-03, -5.91798425e-02, -1.19332066e-02,
       -1.45981964e-02, -2.36388147e-02, -4.89224792e-02,  2.48159561e-02,
        5.96654937e-02,  4.84615862e-02, -4.98594046e-02, -1.48958955e-02,
        5.72910532e-04, -

In [86]:
cos_sim = util.cos_sim(Query, merged_df['doc_vector_pretrained_bert'])


In [109]:
a=cos_sim.argsort()[:][::].tolist()

In [118]:
single_list = []
for sublist in a:
    single_list.extend(sublist)

In [119]:
top_sub = merged_df.iloc[sublist[::-1]]

In [122]:
k=int(input(" No_of docs you want"))
top_sub[:k]

 No_of docs you want 10


,num,name,file_content_chunks,doc_vector_pretrained_bert
166688,9279233,spiderman.no.way.home.(2021).eng.1cd,peter parker best friend come boy come flash t...,"[-0.070744194, -0.10539006, -0.013086834, -0.0..."
89256,9422450,family.guy.s14.e12.scammed.yankees.(2016).eng.1cd,peter go wait let get straight none guy haseve...,"[0.0108731985, -0.047444083, -0.08321992, -0.0..."
166701,9279233,spiderman.no.way.home.(2021).eng.1cd,yeah oh sorry mean peter peter called peter pe...,"[-0.034680914, -0.10299253, -0.007819386, -0.0..."
205958,9346037,heroes.s04.e17.chapter.seventeen.the.wall.(201...,peter previously hero lauren wait second clair...,"[-0.004613507, -0.05195206, 0.009521821, -0.08..."
33724,9418780,family.guy.s12.e15.secondhand.spoke.(2014).eng...,playing peter ready peter okay put peter wow w...,"[-0.04394487, -0.049517408, 0.026066598, -0.06..."
40000,9501633,love.me.s02.e04.episode.2.4.(2023).eng.1cd,wan na dosomething peter start showing upto ap...,"[0.014490434, -0.04982593, 0.022904452, -0.037..."
166690,9279233,spiderman.no.way.home.(2021).eng.1cd,peter owe pleasure right um really sorryto bot...,"[-0.08921423, -0.112601824, -0.037202526, -0.0..."
182176,9422394,family.guy.s11.e18.total.recall.(2013).eng.1cd,peter come time wake got get work ah lois feel...,"[-0.04550732, -0.099507734, 0.0032997, -0.0526..."
41207,9234383,the.real.charlie.chaplin.(2021).eng.1cd,saint peter mean old saint slammed gate right ...,"[0.010897958, 0.025107311, -6.395822e-05, -0.0..."
85909,9420951,family.guy.s08.e09.business.guy.(2009).eng.1cd,sir farting lois could youlet idiot husbandtak...,"[-0.006053679, -0.07671408, -0.0367564, -0.077..."


In [ ]:
merged_df.to_csv("Sub_Titles_with_32.csv",index=False)

In [1]:
import pandas as pd


In [2]:
df = pd.read_csv("Sub_Titles_with_32.csv")

In [3]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

In [4]:
df

,num,name,file_content_chunks,doc_vector_pretrained_bert
0,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,seems todaythat see violence moviesand sex tv ...,[-2.47692876e-02 -7.96079710e-02 1.73655851e-...
1,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,bike hi help u tryingto locate lost big wh ins...,[-1.97190251e-02 -4.68211100e-02 2.28516981e-...
2,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,quiz gon na give name tell nameof yankee candl...,[-2.15782207e-02 -1.04329161e-01 7.22856447e-...
3,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,brand feel isappropriate compensation et ceter...,[-2.22125389e-02 -1.14492767e-01 5.17602414e-...
4,9428782,family.guy.s17.e17.island.adventure.(2019).eng...,thatif sigh right kind took advantage owe apol...,[-2.12117434e-02 -7.87330940e-02 1.88117940e-...
...,...,...,...,...
226170,9521232,workin.moms.s07.e08.fast.and.soft.(2023).eng.1cd,r right away oh good good well great news good...,[ 2.44412757e-02 -3.26235481e-02 -4.25330177e-...
226171,9521232,workin.moms.s07.e08.fast.and.soft.(2023).eng.1cd,harder jenny moaning gena gary gary gary like ...,[-9.17102918e-02 -1.16860144e-01 7.26154894e-...
226172,9521232,workin.moms.s07.e08.fast.and.soft.(2023).eng.1cd,house need help could come get rescue got full...,[-3.42741497e-02 -5.95224462e-02 2.07188819e-...
226173,9521232,workin.moms.s07.e08.fast.and.soft.(2023).eng.1cd,greedy kate care isbeing first market whether ...,[-9.39792544e-02 -2.62093879e-02 6.09866790e-...


In [5]:
User_query = pd.Series([input("Enter a user Query " )])

Enter a user Query  peter parker


In [6]:
def clean_content(text):
    text = re.sub(timestamp_pattern, '', text)  # Remove timestamps
    text = re.sub(unwanted_pattern, '', text)    # Remove unwanted characters
    text = re.sub(r'<.*?>', '', text)            # Remove HTML tags
    text = text.strip()                          # Strip leading/trailing whitespace
    text = re.sub(r'^\d+', '', text.strip())     # Remove leading '1'
    return text.strip()

In [7]:
def preprocess(raw_text):
    # Removing special characters and digits
    sentence = re.sub("[^a-zA-Z]", " ", raw_text)

    #change sentence to lower case
    sentence = sentence.lower()

    # tokenize into words
    tokens =nltk.word_tokenize(sentence)

    #Lemmatize
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    #remove stop words
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in lemmatized_tokens if word.lower() not in stop_words]

    #Join and return
    return " ".join(filtered_tokens)

In [12]:
from tqdm import tqdm
tqdm.pandas()
## step3: Data Preprocessing on train data
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
# Regular expression pattern to remove timestamps
timestamp_pattern = r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}'
# Regular expression pattern to remove unwanted characters like '\r', '\n', 'ï»¿'
unwanted_pattern = r'[\r\nï»¿]'

import re
User_query.progress_apply(lambda x:  clean_content(x)) 

User_query.progress_apply(lambda x:  preprocess(x)) 

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.71s/it]


0    peter parker
dtype: object

In [13]:
Query = User_query.progress_apply(model.encode)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.41s/it]


In [16]:
    df['doc_vector_pretrained_bert'] = df['doc_vector_pretrained_bert'].str.replace('[', '').str.replace(']', '')

C:\Users\Hello\AppData\Local\Temp\ipykernel_21912\3369899128.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['doc_vector_pretrained_bert'] = df['doc_vector_pretrained_bert'].str.replace('[', '').str.replace(']', '')


In [20]:
# Assuming df is your DataFrame and the column name is 'doc_vector_pretrained_bert'
df['doc_vector_pretrained_bert'] = df['doc_vector_pretrained_bert'].progress_apply(lambda x: [float(i) for i in x.split()])


100%|████████████████████████████████████████████████████████████████████████| 226175/226175 [01:37<00:00, 2312.43it/s]


In [21]:
cos_sim = util.cos_sim(Query, df['doc_vector_pretrained_bert'])


In [22]:
cos_sim

tensor([[ 0.0755,  0.0115,  0.0074,  ..., -0.0108, -0.0044,  0.0570]])

In [23]:
a=cos_sim.argsort()[:][::].tolist()

In [24]:
single_list = []
for sublist in a:
    single_list.extend(sublist)

In [25]:
top_sub = df.iloc[sublist[::-1]]

In [26]:
top_sub

,num,name,file_content_chunks,doc_vector_pretrained_bert
166688,9279233,spiderman.no.way.home.(2021).eng.1cd,peter parker best friend come boy come flash t...,"[-0.0707441941, -0.105390057, -0.013086834, -0..."
166699,9279233,spiderman.no.way.home.(2021).eng.1cd,may sorry sorry sorry love parker come hand co...,"[-0.0371851139, -0.138281539, -0.0327688344, -..."
54437,9235781,spiderman.no.way.home.(2021).eng.1cd,parker stuff know hm uh little complicated oh ...,"[-0.0461160839, -0.109720223, -0.0389870889, -..."
166701,9279233,spiderman.no.way.home.(2021).eng.1cd,yeah oh sorry mean peter peter called peter pe...,"[-0.0346809141, -0.102992527, -0.0078193862, -..."
166690,9279233,spiderman.no.way.home.(2021).eng.1cd,peter owe pleasure right um really sorryto bot...,"[-0.0892142281, -0.112601824, -0.0372025259, -..."
...,...,...,...,...
101919,9203358,orphan.first.kill.(2022).eng.1cd,unit highly functioning emotionally well voice...,"[-0.0438047983, -0.119256467, -0.00224961014, ..."
151987,9357951,trafficked.with.mariana.van.zeller.s01.e02.fen...,beatriz agrees let mefollow border car parked ...,"[-0.0366659909, -0.0138048632, -0.0061825607, ..."
173244,9281177,doc.martin.s10.e07.love.will.set.you.free.(202...,wa trying help trying get helen long get er mi...,"[-0.0958784893, -0.0230194237, 0.0512975156, 0..."
98625,9267628,orphan.first.kill.(2022).eng.1cd,know sorry wa fault esther wandered thestore p...,"[-0.00756117934, -0.0959375799, 0.0251081865, ..."


In [27]:
k=int(input(" No_of docs you want"))
top_sub[:k]

 No_of docs you want 10


,num,name,file_content_chunks,doc_vector_pretrained_bert
166688,9279233,spiderman.no.way.home.(2021).eng.1cd,peter parker best friend come boy come flash t...,"[-0.0707441941, -0.105390057, -0.013086834, -0..."
166699,9279233,spiderman.no.way.home.(2021).eng.1cd,may sorry sorry sorry love parker come hand co...,"[-0.0371851139, -0.138281539, -0.0327688344, -..."
54437,9235781,spiderman.no.way.home.(2021).eng.1cd,parker stuff know hm uh little complicated oh ...,"[-0.0461160839, -0.109720223, -0.0389870889, -..."
166701,9279233,spiderman.no.way.home.(2021).eng.1cd,yeah oh sorry mean peter peter called peter pe...,"[-0.0346809141, -0.102992527, -0.0078193862, -..."
166690,9279233,spiderman.no.way.home.(2021).eng.1cd,peter owe pleasure right um really sorryto bot...,"[-0.0892142281, -0.112601824, -0.0372025259, -..."
166684,9279233,spiderman.no.way.home.(2021).eng.1cd,hi guy um tom holland joined today tobey magui...,"[-0.0486545898, -0.0953649357, -0.0649602264, ..."
166685,9279233,spiderman.no.way.home.(2021).eng.1cd,okay yeah yeah yeah come come come okay sorry ...,"[-0.0534517169, -0.163306788, -0.0183013771, -..."
54425,9235781,spiderman.no.way.home.(2021).eng.1cd,get hang new ground beingbroken need eight hou...,"[-0.0638472736, -0.131348252, -0.0330801755, -..."
184198,9278516,spiderman.no.way.home.(2021).eng.1cd,mj peter hey mj nothingto sir say anythingwith...,"[-0.0327948667, -0.0514463708, -0.0455897711, ..."
40022,9452963,parker.lewis.cant.lose.s03.e01.flamingo.graffi...,point love poked way throughall trouble said p...,"[-0.0898734704, -0.00354854972, -0.045404084, ..."


In [34]:
top_sub.reset_index()

,index,num,name,file_content_chunks,doc_vector_pretrained_bert
0,166688,9279233,spiderman.no.way.home.(2021).eng.1cd,peter parker best friend come boy come flash t...,"[-0.0707441941, -0.105390057, -0.013086834, -0..."
1,166699,9279233,spiderman.no.way.home.(2021).eng.1cd,may sorry sorry sorry love parker come hand co...,"[-0.0371851139, -0.138281539, -0.0327688344, -..."
2,54437,9235781,spiderman.no.way.home.(2021).eng.1cd,parker stuff know hm uh little complicated oh ...,"[-0.0461160839, -0.109720223, -0.0389870889, -..."
3,166701,9279233,spiderman.no.way.home.(2021).eng.1cd,yeah oh sorry mean peter peter called peter pe...,"[-0.0346809141, -0.102992527, -0.0078193862, -..."
4,166690,9279233,spiderman.no.way.home.(2021).eng.1cd,peter owe pleasure right um really sorryto bot...,"[-0.0892142281, -0.112601824, -0.0372025259, -..."
...,...,...,...,...,...
226170,101919,9203358,orphan.first.kill.(2022).eng.1cd,unit highly functioning emotionally well voice...,"[-0.0438047983, -0.119256467, -0.00224961014, ..."
226171,151987,9357951,trafficked.with.mariana.van.zeller.s01.e02.fen...,beatriz agrees let mefollow border car parked ...,"[-0.0366659909, -0.0138048632, -0.0061825607, ..."
226172,173244,9281177,doc.martin.s10.e07.love.will.set.you.free.(202...,wa trying help trying get helen long get er mi...,"[-0.0958784893, -0.0230194237, 0.0512975156, 0..."
226173,98625,9267628,orphan.first.kill.(2022).eng.1cd,know sorry wa fault esther wandered thestore p...,"[-0.00756117934, -0.0959375799, 0.0251081865, ..."


In [36]:
top_sub_top = top_sub.reset_index()

In [45]:
top_sub_top.loc[0,'file_content_chunks']

'peter parker best friend come boy come flash thompson wan na readabout inspiring friendship new book flashpoint one spider two heart million crazy memory check go got ta wait go see inside okay keep watching midtown news year bring closeand personal coverage peter fighting biggest battle yet college admission peter would love welcome youback midtown high shape hero uh huh murderer stop huh honour serve sir mysterio wa right stop mysterio wa right student put together helped little bit tried stop youso many time powered great job hope time check feel free walkor swing hallway crawl ceilingto avoid everybody know gon na know stop embarrassing conspiracy theorist lab partner know climb climb climb climb look sticky bastard go freak getsall best idea call spidey mentor peter good see bud see later oh god cute likebeing spider man teacher well peter always beenmore student like son also little brother think crazy mysterio conspiracy theoriesthat spreading online conspiracy theory justin ti